In [1]:
## Preprocessing

# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
app_df = pd.read_csv("charity_data.csv")
app_df.head()


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
df = app_df.drop(columns=['EIN','NAME'])

df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [3]:
# Determine the number of unique values in each column.
df.nunique()

# Look at APPLICATION_TYPE value counts for binning
application_types = df['APPLICATION_TYPE'].value_counts()
application_types

# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = [index for index,row in application_types.items() if row < 500]

# Replace in dataframe
for app in application_types_to_replace:
    df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
df['APPLICATION_TYPE'].value_counts()

# Look at CLASSIFICATION value counts for binning
#  YOUR CODE GOES HEREclassification_types = df['CLASSIFICATION'].value_counts()
classification_types = df['CLASSIFICATION'].value_counts()
classification_types


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [4]:
# You may find it helpful to look at CLASSIFICATION value counts >1
[[index,row] for index,row in classification_types.items() if row > 1]

# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = [index for index,row in classification_types.items() if row < classification_types.mean()]

# Replace in dataframe
for cls in classifications_to_replace:
    app_df['CLASSIFICATION'] = app_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
app_df['CLASSIFICATION'].value_counts()

# Convert categorical data to numeric with `pd.get_dummies`
dummies = pd.get_dummies(df[['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
       'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS']])
processed_df = pd.concat([df[['STATUS', 'ASK_AMT']], dummies], axis=1)

# Split our preprocessed data into our features and target arrays
X = processed_df
y = df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=.2,
                                                    random_state=42)

print(f"X_train Shape: {X_train.shape}, X_test Shape: {X_test.shape}")


X_train Shape: (27439, 108), X_test Shape: (6860, 108)


In [5]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [6]:
model = tf.keras.models.load_model('optimized_model2.h5')

In [ ]:
# Compile the model
model.compile(loss="BinaryCrossentropy", optimizer="adam", metrics=['accuracy', 'mse'])

# Train the model
fit_model = model.fit(X_train_scaled, y_train, epochs=100, validation_data=(X_test_scaled, y_test))


Epoch 1/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5408 - accuracy: 0.7343 - mse: 0.1815 - val_loss: 0.5564 - val_accuracy: 0.7290 - val_mse: 0.1868
Epoch 2/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5408 - accuracy: 0.7347 - mse: 0.1815 - val_loss: 0.5553 - val_accuracy: 0.7292 - val_mse: 0.1865
Epoch 3/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5408 - accuracy: 0.7342 - mse: 0.1815 - val_loss: 0.5554 - val_accuracy: 0.7278 - val_mse: 0.1865
Epoch 4/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5407 - accuracy: 0.7350 - mse: 0.1815 - val_loss: 0.5553 - val_accuracy: 0.7290 - val_mse: 0.1864
Epoch 5/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5405 - accuracy: 0.7348 - mse: 0.1814 - val_loss: 0.5562 - val_accuracy: 0.7290 - val_mse: 0.1868
Epoch 6/100
858/858 [==============================] - 2s 2ms/step - loss: 0.5405 - accuracy: 0.7346 - mse: 0.1815 - val_loss

858/858 [==============================] - 1s 2ms/step - loss: 0.5399 - accuracy: 0.7362 - mse: 0.1812 - val_loss: 0.5565 - val_accuracy: 0.7280 - val_mse: 0.1868
Epoch 48/100
858/858 [==============================] - 1s 2ms/step - loss: 0.5398 - accuracy: 0.7359 - mse: 0.1812 - val_loss: 0.5566 - val_accuracy: 0.7294 - val_mse: 0.1868
Epoch 49/100
858/858 [==============================] - 1s 2ms/step - loss: 0.5398 - accuracy: 0.7364 - mse: 0.1812 - val_loss: 0.5560 - val_accuracy: 0.7293 - val_mse: 0.1865
Epoch 50/100
858/858 [==============================] - 1s 2ms/step - loss: 0.5397 - accuracy: 0.7359 - mse: 0.1811 - val_loss: 0.5558 - val_accuracy: 0.7292 - val_mse: 0.1865
Epoch 51/100
858/858 [==============================] - 1s 2ms/step - loss: 0.5399 - accuracy: 0.7354 - mse: 0.1812 - val_loss: 0.5563 - val_accuracy: 0.7277 - val_mse: 0.1867
Epoch 52/100
858/858 [==============================] - 1s 2ms/step - loss: 0.5398 - accuracy: 0.7365 - mse: 0.1812 - val_loss: 0.555

In [ ]:
# Evaluate the model and visualize performance on the test set
evaluate_metrics = model.evaluate(X_test_scaled, y_test)
print(f"Test Loss: {evaluate_metrics[0]}, Test Accuracy: {evaluate_metrics[1]}")


# Save the optimized model
model.save('optimized_model2.h5')